In [ ]:
#The below code is used to access files stored in your Google Drive and work with them in your Colab notebook. You can read, write, and manipulate files in your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/Natural Language Processing/Projectme/test.csv'
# Read the csv file
df = pd.read_csv(file_path)
df.head()

,id,dialogue,summary,topic
0,test_0_1,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method
1,test_0_2,"#Person1#: Ms. Dawson, I need you to take a di...",In order to prevent employees from wasting tim...,company policy
2,test_0_3,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for #Person1# abo...,dictation
3,test_1_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation
4,test_1_2,#Person1#: You're finally here! What took so l...,#Person2# decides to follow #Person1#'s sugges...,transportation


In [ ]:
import pandas as pd
import nltk
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Download required NLTK resources
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

def lemmatize_words(words):
    return [wordlemmatizer.lemmatize(word) for word in words]

def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    return re.sub(regex, '', text)

def freq(words):
    words = [word.lower() for word in words]
    return {word: words.count(word) for word in set(words)}

def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    return [word for word, tag in pos_tag if tag.startswith("NN") or tag.startswith("VB")]

def tf_score(word, sentence):
    len_sentence = len(sentence.split())
    return sentence.split().count(word) / len_sentence

def idf_score(no_of_sentences, word, sentences):
    no_of_sentence_containing_word = sum(word in sentence.split() for sentence in sentences)
    if no_of_sentence_containing_word == 0:
        return 0  # Avoid division by zero
    return math.log10(no_of_sentences / no_of_sentence_containing_word)

def word_tfidf(dict_freq, word, sentences, sentence):
    tf = tf_score(word, sentence)
    idf = idf_score(len(sentences), word, sentences)
    return tf * idf

def sentence_importance(sentence, dict_freq, sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence))
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
        if word.lower() not in Stopwords and len(word) > 1:
            word = word.lower()
            word = wordlemmatizer.lemmatize(word)
            sentence_score += word_tfidf(dict_freq, word, sentences, sentence)
    return sentence_score


In [ ]:
def clean_conversation(conversation):
    # Remove instruction tags completely
    conversation = re.sub(r'<s>\[INST\].*?\[\/INST\]</s>', '', conversation)

    # Remove additional tags if they exist
    conversation = re.sub(r'<s>.*?</s>', '', conversation)

    # Remove speaker labels
    conversation = re.sub(r'(Person \d+:|Customer:)', '', conversation)

    return conversation.strip()

def summarize_conversation(conversation, summary_percentage):
    conversation = clean_conversation(conversation)
    tokenized_sentence = sent_tokenize(conversation)
    processed_text = remove_special_characters(conversation)
    processed_text = re.sub(r'\d+', '', processed_text)
    tokenized_words = word_tokenize(processed_text)
    tokenized_words = [word for word in tokenized_words if word.lower() not in Stopwords and len(word) > 1]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)

    sentence_scores = {sent: sentence_importance(sent, word_freq, tokenized_sentence) for sent in tokenized_sentence}
    no_of_sentences = int((summary_percentage * len(tokenized_sentence)) / 100)
    important_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:no_of_sentences]

    summary = " ".join(sorted(important_sentences, key=lambda s: tokenized_sentence.index(s)))
    return summary


In [ ]:
summary_percentage = 20  # Example: 20%
df['summary_tfidf'] = df['dialogue'].apply(lambda x: summarize_conversation(x, summary_percentage))

In [ ]:
df.head()

,id,dialogue,summary,topic,summary_tfidf
0,test_0_1,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method,"Effective immediately, all office communicatio..."
1,test_0_2,"#Person1#: Ms. Dawson, I need you to take a di...",In order to prevent employees from wasting tim...,company policy,"Effective immediately, all office communicatio..."
2,test_0_3,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for #Person1# abo...,dictation,"Effective immediately, all office communicatio..."
3,test_1_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation,#Person2#: I got stuck in traffic again. Maybe...
4,test_1_2,#Person1#: You're finally here! What took so l...,#Person2# decides to follow #Person1#'s sugges...,transportation,#Person2#: I got stuck in traffic again. Maybe...


In [ ]:
print(df['dialogue'].iloc[0])

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh

In [ ]:
print(df['summary'].iloc[0])

Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.


In [ ]:
print(df['summary_tfidf'].iloc[0])

Effective immediately, all office communications are restricted to email correspondence and official memos. Now, please continue with the memo. Any employee who persists in using Instant Messaging will first receive a warning and be placed on probation. Any questions regarding this new policy may be directed to department heads. Please get this memo typed up and distributed to all employees before 4 pm.


In [ ]:
!pip install rouge-score
import pandas as pd
from rouge_score import rouge_scorer

# Assuming df is your DataFrame

# Function to calculate ROUGE scores
def calculate_rouge(row):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(row['summary'], row['summary_tfidf'])
    return scores

# Apply the function to each row
df['rouge_scores'] = df.apply(calculate_rouge, axis=1)

# Display the first few rows to see the ROUGE scores
print(df[['id', 'rouge_scores']].head())
!

         id                                       rouge_scores
0  test_0_1  {'rouge1': (0.19047619047619047, 0.44444444444...
1  test_0_2  {'rouge1': (0.23809523809523808, 0.41666666666...
2  test_0_3  {'rouge1': (0.1111111111111111, 0.259259259259...
3  test_1_1  {'rouge1': (0.13953488372093023, 0.27272727272...
4  test_1_2  {'rouge1': (0.23255813953488372, 0.52631578947...


In [ ]:
df.head()

,id,dialogue,summary,topic,summary_tfidf,rouge_scores
0,test_0_1,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson helps #Person1# to write a memo to ...,communication method,"Effective immediately, all office communicatio...","{'rouge1': (0.19047619047619047, 0.44444444444..."
1,test_0_2,"#Person1#: Ms. Dawson, I need you to take a di...",In order to prevent employees from wasting tim...,company policy,"Effective immediately, all office communicatio...","{'rouge1': (0.23809523809523808, 0.41666666666..."
2,test_0_3,"#Person1#: Ms. Dawson, I need you to take a di...",Ms. Dawson takes a dictation for #Person1# abo...,dictation,"Effective immediately, all office communicatio...","{'rouge1': (0.1111111111111111, 0.259259259259..."
3,test_1_1,#Person1#: You're finally here! What took so l...,#Person2# arrives late because of traffic jam....,public transportation,#Person2#: I got stuck in traffic again. Maybe...,"{'rouge1': (0.13953488372093023, 0.27272727272..."
4,test_1_2,#Person1#: You're finally here! What took so l...,#Person2# decides to follow #Person1#'s sugges...,transportation,#Person2#: I got stuck in traffic again. Maybe...,"{'rouge1': (0.23255813953488372, 0.52631578947..."


In [ ]:
print(df['rouge_scores'].iloc[0])

{'rouge1': Score(precision=0.19047619047619047, recall=0.4444444444444444, fmeasure=0.26666666666666666), 'rouge2': Score(precision=0.03225806451612903, recall=0.07692307692307693, fmeasure=0.045454545454545456), 'rougeL': Score(precision=0.09523809523809523, recall=0.2222222222222222, fmeasure=0.13333333333333333)}


In [ ]:
# Initialize accumulators for ROUGE scores
total_rouge1 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rouge2 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rougeL = {'precision': 0, 'recall': 0, 'fmeasure': 0}
num_rows = len(df)

# Accumulate scores from each row
for index, row in df.iterrows():
    total_rouge1['precision'] += row['rouge_scores']['rouge1'][0]
    total_rouge1['recall'] += row['rouge_scores']['rouge1'][1]
    total_rouge1['fmeasure'] += row['rouge_scores']['rouge1'][2]

    total_rouge2['precision'] += row['rouge_scores']['rouge2'][0]
    total_rouge2['recall'] += row['rouge_scores']['rouge2'][1]
    total_rouge2['fmeasure'] += row['rouge_scores']['rouge2'][2]

    total_rougeL['precision'] += row['rouge_scores']['rougeL'][0]
    total_rougeL['recall'] += row['rouge_scores']['rougeL'][1]
    total_rougeL['fmeasure'] += row['rouge_scores']['rougeL'][2]

# Calculate averages
avg_rouge1 = {k: v / num_rows for k, v in total_rouge1.items()}
avg_rouge2 = {k: v / num_rows for k, v in total_rouge2.items()}
avg_rougeL = {k: v / num_rows for k, v in total_rougeL.items()}

# Display average scores
print("Average ROUGE-1 Scores:", avg_rouge1)
print("Average ROUGE-2 Scores:", avg_rouge2)
print("Average ROUGE-L Scores:", avg_rougeL)


Average ROUGE-1 Scores: {'precision': 0.20313408066967534, 'recall': 0.29976508368077, 'fmeasure': 0.22909214852985244}
Average ROUGE-2 Scores: {'precision': 0.05322613527020472, 'recall': 0.0772612635758411, 'fmeasure': 0.059688496182405106}
Average ROUGE-L Scores: {'precision': 0.16075127540736406, 'recall': 0.23030085717294557, 'fmeasure': 0.17865877667930097}


In [ ]:
import pandas as pd
file_path = '/content/drive/My Drive/Natural Language Processing/Projectme/validation.csv'
# Read the csv file
df = pd.read_csv(file_path)
df.head()

,id,dialogue,summary,topic
0,dev_0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor
1,dev_1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise
2,dev_2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods
3,dev_3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens
4,dev_4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school


In [ ]:
import pandas as pd
import nltk
import re
import math
import operator
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

# Download required NLTK resources
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

def lemmatize_words(words):
    return [wordlemmatizer.lemmatize(word) for word in words]

def remove_special_characters(text):
    regex = r'[^a-zA-Z0-9\s]'
    return re.sub(regex, '', text)

def freq(words):
    words = [word.lower() for word in words]
    return {word: words.count(word) for word in set(words)}

def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    return [word for word, tag in pos_tag if tag.startswith("NN") or tag.startswith("VB")]

def tf_score(word, sentence):
    len_sentence = len(sentence.split())
    return sentence.split().count(word) / len_sentence

def idf_score(no_of_sentences, word, sentences):
    no_of_sentence_containing_word = sum(word in sentence.split() for sentence in sentences)
    if no_of_sentence_containing_word == 0:
        return 0  # Avoid division by zero
    return math.log10(no_of_sentences / no_of_sentence_containing_word)

def word_tfidf(dict_freq, word, sentences, sentence):
    tf = tf_score(word, sentence)
    idf = idf_score(len(sentences), word, sentences)
    return tf * idf

def sentence_importance(sentence, dict_freq, sentences):
    sentence_score = 0
    sentence = remove_special_characters(str(sentence))
    sentence = re.sub(r'\d+', '', sentence)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
        if word.lower() not in Stopwords and len(word) > 1:
            word = word.lower()
            word = wordlemmatizer.lemmatize(word)
            sentence_score += word_tfidf(dict_freq, word, sentences, sentence)
    return sentence_score


In [ ]:

def clean_conversation(conversation):
    # Remove instruction tags completely
    conversation = re.sub(r'<s>\[INST\].*?\[\/INST\]</s>', '', conversation)

    # Remove additional tags if they exist
    conversation = re.sub(r'<s>.*?</s>', '', conversation)

    # Remove speaker labels including "Person" and "#" symbols
    conversation = re.sub(r'(#Person \d+#:|#Customer#:)', '', conversation)

    return conversation.strip()

def summarize_conversation(conversation, summary_percentage):
    conversation = clean_conversation(conversation)
    tokenized_sentence = sent_tokenize(conversation)
    processed_text = remove_special_characters(conversation)
    processed_text = re.sub(r'\d+', '', processed_text)
    tokenized_words = word_tokenize(processed_text)
    tokenized_words = [word for word in tokenized_words if word.lower() not in Stopwords and len(word) > 1]
    tokenized_words = lemmatize_words(tokenized_words)
    word_freq = freq(tokenized_words)

    sentence_scores = {sent: sentence_importance(sent, word_freq, tokenized_sentence) for sent in tokenized_sentence}
    no_of_sentences = int((summary_percentage * len(tokenized_sentence)) / 100)
    important_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:no_of_sentences]

    summary = " ".join(sorted(important_sentences, key=lambda s: tokenized_sentence.index(s)))
    return summary


In [ ]:
summary_percentage = 20  # Example: 20%
df['summary_tfidf'] = df['dialogue'].apply(lambda x: summarize_conversation(x, summary_percentage))

In [ ]:
df.head()

,id,dialogue,summary,topic,summary_tfidf
0,dev_0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor,#Person2#: I ' Ve been having trouble breathin...
1,dev_1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise,What time do you want to go? You're messing ev...
2,dev_2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods,I've started eating better myself. #Person1#: ...
3,dev_3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens,"They are called UFOs, so not everybody can see..."
4,dev_4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school,"#Person1#: Well anyway, do you plan on going t..."


In [ ]:
print(df['dialogue'].iloc[0])

#Person1#: Hello, how are you doing today?
#Person2#: I ' Ve been having trouble breathing lately.
#Person1#: Have you had any type of cold lately?
#Person2#: No, I haven ' t had a cold. I just have a heavy feeling in my chest when I try to breathe.
#Person1#: Do you have any allergies that you know of?
#Person2#: No, I don ' t have any allergies that I know of.
#Person1#: Does this happen all the time or mostly when you are active?
#Person2#: It happens a lot when I work out.
#Person1#: I am going to send you to a pulmonary specialist who can run tests on you for asthma.
#Person2#: Thank you for your help, doctor.


In [ ]:
print(df['summary'].iloc[0])

#Person2# has trouble breathing. The doctor asks #Person2# about it and will send #Person2# to a pulmonary specialist.


In [ ]:
print(df['summary_tfidf'].iloc[0])

#Person2#: I ' Ve been having trouble breathing lately. #Person2#: It happens a lot when I work out.


In [ ]:
!pip install rouge-score
import pandas as pd
from rouge_score import rouge_scorer

# Assuming df is your DataFrame

# Function to calculate ROUGE scores
def calculate_rouge(row):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(row['summary'], row['summary_tfidf'])
    return scores

# Apply the function to each row
df['rouge_scores'] = df.apply(calculate_rouge, axis=1)

# Display the first few rows to see the ROUGE scores
print(df[['id', 'rouge_scores']].head())
!

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=27109ae1dbb1a57a10d034cf04449f25023bdace816b919f83f6cd2789e0ac16
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
      id                                       rouge_scores
0  dev_0  {'rouge1': (0.35294117647058826, 0.33333333333...
1  dev_1  {'rouge1': (0.08695652173913043, 0.125, 0.1025...
2  dev_2  {'rouge1': (0.13333333333333333, 0.125, 0.1290...
3  dev_3  {'rouge1': (0.16666666666666666, 0.25925925925...
4  dev_4  {'rouge1': (0.4782608695652174, 0.611111111111...


In [ ]:
df.head()

,id,dialogue,summary,topic,summary_tfidf,rouge_scores
0,dev_0,"#Person1#: Hello, how are you doing today?\n#P...",#Person2# has trouble breathing. The doctor as...,see a doctor,#Person2#: I ' Ve been having trouble breathin...,"{'rouge1': (0.35294117647058826, 0.33333333333..."
1,dev_1,#Person1#: Hey Jimmy. Let's go workout later t...,#Person1# invites Jimmy to go workout and pers...,do exercise,What time do you want to go? You're messing ev...,"{'rouge1': (0.08695652173913043, 0.125, 0.1025..."
2,dev_2,#Person1#: I need to stop eating such unhealth...,#Person1# plans to stop eating unhealthy foods...,healthy foods,I've started eating better myself. #Person1#: ...,"{'rouge1': (0.13333333333333333, 0.125, 0.1290..."
3,dev_3,#Person1#: Do you believe in UFOs?\n#Person2#:...,#Person2# believes in UFOs and can see them in...,UFOs and aliens,"They are called UFOs, so not everybody can see...","{'rouge1': (0.16666666666666666, 0.25925925925..."
4,dev_4,#Person1#: Did you go to school today?\n#Perso...,#Person1# didn't go to school today. #Person2#...,go to school,"#Person1#: Well anyway, do you plan on going t...","{'rouge1': (0.4782608695652174, 0.611111111111..."


In [ ]:
print(df['rouge_scores'].iloc[0])

{'rouge1': Score(precision=0.35294117647058826, recall=0.3333333333333333, fmeasure=0.34285714285714286), 'rouge2': Score(precision=0.0625, recall=0.058823529411764705, fmeasure=0.06060606060606061), 'rougeL': Score(precision=0.35294117647058826, recall=0.3333333333333333, fmeasure=0.34285714285714286)}


In [ ]:
# Initialize accumulators for ROUGE scores
total_rouge1 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rouge2 = {'precision': 0, 'recall': 0, 'fmeasure': 0}
total_rougeL = {'precision': 0, 'recall': 0, 'fmeasure': 0}
num_rows = len(df)

# Accumulate scores from each row
for index, row in df.iterrows():
    total_rouge1['precision'] += row['rouge_scores']['rouge1'][0]
    total_rouge1['recall'] += row['rouge_scores']['rouge1'][1]
    total_rouge1['fmeasure'] += row['rouge_scores']['rouge1'][2]

    total_rouge2['precision'] += row['rouge_scores']['rouge2'][0]
    total_rouge2['recall'] += row['rouge_scores']['rouge2'][1]
    total_rouge2['fmeasure'] += row['rouge_scores']['rouge2'][2]

    total_rougeL['precision'] += row['rouge_scores']['rougeL'][0]
    total_rougeL['recall'] += row['rouge_scores']['rougeL'][1]
    total_rougeL['fmeasure'] += row['rouge_scores']['rougeL'][2]

# Calculate averages
avg_rouge1 = {k: v / num_rows for k, v in total_rouge1.items()}
avg_rouge2 = {k: v / num_rows for k, v in total_rouge2.items()}
avg_rougeL = {k: v / num_rows for k, v in total_rougeL.items()}

# Display average scores
print("Average ROUGE-1 Scores:", avg_rouge1)
print("Average ROUGE-2 Scores:", avg_rouge2)
print("Average ROUGE-L Scores:", avg_rougeL)


Average ROUGE-1 Scores: {'precision': 0.23173441072817288, 'recall': 0.2880282383262665, 'fmeasure': 0.24602134775004522}
Average ROUGE-2 Scores: {'precision': 0.06771219022249961, 'recall': 0.08153520507075922, 'fmeasure': 0.07069662937330834}
Average ROUGE-L Scores: {'precision': 0.1878009524981344, 'recall': 0.22737551510067514, 'fmeasure': 0.1965640966103361}
